SIN-392 - Introdução ao Processamento Digital de Imagens (2023-1)

# Aula 17 - Redes neurais convolucionais

Prof. João Fernando Mari ([*joaofmari.github.io*](https://joaofmari.github.io/))

---

## Montando o Google Drive
---

* Caso esteja executanto no Google Colab, não esquecer de habilitar o acesso à GPU.
    * Editar >> Configurações de notebook >> Acelerador de hardware
    * Selecione GPU
    * OK
* Após o uso, desabilitar o acesso.

In [1]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

# DEBUG
print(IN_COLAB)

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

False


## Importando as bibliotecas
---

In [2]:
import numpy as np

from sklearn import metrics
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

### %matplotlib notebook

## Checking GPU Access
---

In [3]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('\nDevice: {0}'.format(DEVICE))


Device: cuda


In [4]:
!nvidia-smi

Tue Oct  3 17:49:48 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.113.01             Driver Version: 535.113.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1050 Ti     Off | 00000000:01:00.0 Off |                  N/A |
| 46%   31C    P8              N/A /  75W |      8MiB /  4096MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Settings for reproducibility
---

In [5]:
np.random.seed(1234)

## Definindo alguns hiperparametros
---

In [6]:
# Tamanho do mini-lote (mini-batch)
batch_size = 8 

# Number of training epochs
max_epochs = 50 # 20

## O conjunto de imagens (dataset)
---

* Neste primeiro exemplo, usaremos o dataset CIFAR-10.
* O CIFAR-10 é composto por 60000 imagens com tamanho 32 x 32 organizadas em 10 classes, com 6000 imagens por classe.
    * Classes: 'plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'
* O CIFAR-10 já vem divido em um conjunto de treino com 50000 imagens e um conjunto de testes com 10000.
    * Ou seja, caso necessário deve-se separar uma parte do conjunto de treinamento para ser utilizado como validação.
* O CIFAR-10 pode ser acessado diretamente a partir da biblioteca torchvision.
    * Ou seja, não é necessário carregar o conjunto de dados a partir do armazenamento local.

In [7]:
# Nomes das classes
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [8]:
# Define uma sequencia de transformações que serão aplicadas sobre as imagens dos datasets
transform = transforms.Compose([# Converte o mini-lote para tensor. Automaticamente converte valores para faixa [0, 1]
                                transforms.ToTensor(), 
                                # Converte os valores para a faixa [-1, 1]
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) 

# Datasets
# --------
# Conjunto de treinamento
dataset_train = torchvision.datasets.CIFAR10(root='./data', 
                                             train=True,
                                             download=True, 
                                             transform=transform)
# Conjunto de testes
dataset_test = torchvision.datasets.CIFAR10(root='./data', 
                                            train=False,
                                            download=True, 
                                            transform=transform)

# Dataloaders
# -----------
# Conjunto de treinamento
dataloader_train = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size,
                                               shuffle=True, num_workers=2)
# Conjunto de testes
dataloader_test = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size,
                                              shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


## Building a simple Convolutional Neural Network
---

* Vamos construir uma rede neural convolucional simples do zero usando a biblioteca PyTorch.
* A nossa CNN possui a seguinte estrutura:
    1. Camada convolucional 1: Entradas com 3 canais e 6 filtros com tamanho 5 x 5
    2. Função de ativação ReLU
    3. Camada de Max Pooling
    4. Camada convolucional 2: Entrada com 6 canais e 16 filtros com tamanho 5 x 5
    2. Função de ativação ReLU
    3. Camada de Max Pooling
    4. Achatamento das saídas
    5. Camada completamente conectada com 120 neurônios
    6. Função de ativação ReLU
    7. Camada completamente conectada com 84 neurônios
    8. Função de ativação ReLU
    9. Camada de saída com 10 neurônios (dataset com 10 classes).

In [9]:
class Net(nn.Module):
    """
    Considering each image having 32 rows by 32 columns:
    
    Input [3, 32, 32] 
    Conv1(3, 6, 5) [6, 28, 28] 
    Pool(2, 2) [6, 14, 14] 
    Conv2(6, 16, 5) [16, 10, 10]
    Pool(2, 2) [16, 5, 5]
    Flatten [400] (16 x 5 x 5 = 400)
    Fc1 [120]
    Fc2 [84]
    Fc3 [10]
    """
    def __init__(self):
        """
        torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, 
                        dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)
        torch.nn.MaxPool2d(kernel_size, stride=None, padding=0, dilation=1, return_indices=False, ceil_mode=False)
        torch.nn.Linear(in_features, out_features, bias=True, device=None, dtype=None)
        """
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # conv1(x): Input: [32, 32, 3]. Output: [28, 28, 6].
        # - Como padding=0 e kernel_size=5, a imagem é "reduzida" 2 linhas (5-1/2 = 2) acima e abaixo e 2 colunas à esquerda e à direita.
        x = self.conv1(x)
        x = F.relu(x)
        # pool: Input: [28, 28, 6], Output: [14, 14, 6]
        x = self.pool(x)
        # conv2: Input: [14, 14, 6]. Output: [10, 10, 16].
        # - Como padding=0 e kernel_size=5, a imagem é "reduzida" 2 linhas (5-1/2 = 2) acima e abaixo e 2 colunas à esquerda e à direita.
        x = self.conv2(x)
        x = F.relu(x)
        # pool: Input: [10, 10, 16], Output: [5, 5, 16]
        x = self.pool(x)
        # flatten: Input: [5, 5, 16]. Output: [400]
        x = torch.flatten(x, 1) 
        # fc1: Input: [400]. Output: [120]
        x = self.fc1(x)
        x = F.relu(x)
        # fc2: Input: [120]. Output: [84]
        x = self.fc2(x)
        x = F.relu(x)
        # fc3: Input: [80]. Output: [num_classes]
        x = self.fc3(x)
        
        return x

In [10]:
# Instância um objeto da classe Net
model = Net()

# Send model to GPU
model = model.cuda() # Cuda

print(model)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


## Loss function and optimizer
---

In [11]:
# Função de perda (loss) - Entrôpia cruzada
criterion = nn.CrossEntropyLoss()

# Otimizador - Stochastic Gradient Descent
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

## Training the model
---

In [12]:
# Itera ao longo do dataset por um número de épocas.
for epoch in range(max_epochs):  

    # Habilita o modelo para o modo de treino 
    model.train() 

    # Perda (loss) nesta época
    train_loss = 0.0

    # Treino
    for i, (inputs, labels) in enumerate(dataloader_train):

        # Send data to GPU
        inputs = inputs.to(DEVICE) 
        labels = labels.to(DEVICE) 

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward 
        outputs = model(inputs)
        
        # Calcula a função de perda
        loss = criterion(outputs, labels)
        
        # Backward
        loss.backward()
        
        # Otimiza os parâmetros (pesos)
        optimizer.step()

        # Update the epoch training loss
        train_loss += loss.item() * inputs.size(0)

    # Print epoch information
    print(f'Epoch {epoch}/{max_epochs - 1} - TRAIN Loss: {train_loss:.4f}')
        
print('\nTreinamento finalizado!')

Epoch 0/49 - TRAIN Loss: 91059.5220
Epoch 1/49 - TRAIN Loss: 71219.0159
Epoch 2/49 - TRAIN Loss: 63322.4090
Epoch 3/49 - TRAIN Loss: 58354.1272
Epoch 4/49 - TRAIN Loss: 54607.4862
Epoch 5/49 - TRAIN Loss: 51305.5750
Epoch 6/49 - TRAIN Loss: 48686.3224
Epoch 7/49 - TRAIN Loss: 46180.8776
Epoch 8/49 - TRAIN Loss: 44208.9232
Epoch 9/49 - TRAIN Loss: 42288.2284
Epoch 10/49 - TRAIN Loss: 40336.2707
Epoch 11/49 - TRAIN Loss: 38699.2404
Epoch 12/49 - TRAIN Loss: 37199.3423
Epoch 13/49 - TRAIN Loss: 35760.1628
Epoch 14/49 - TRAIN Loss: 34489.2443
Epoch 15/49 - TRAIN Loss: 33462.9016
Epoch 16/49 - TRAIN Loss: 32366.0826
Epoch 17/49 - TRAIN Loss: 31418.7436
Epoch 18/49 - TRAIN Loss: 30438.6922
Epoch 19/49 - TRAIN Loss: 29356.8523
Epoch 20/49 - TRAIN Loss: 28557.5266
Epoch 21/49 - TRAIN Loss: 28080.4109
Epoch 22/49 - TRAIN Loss: 27369.6549
Epoch 23/49 - TRAIN Loss: 26697.6281
Epoch 24/49 - TRAIN Loss: 25784.3382
Epoch 25/49 - TRAIN Loss: 25403.9090
Epoch 26/49 - TRAIN Loss: 24871.5800
Epoch 27/49

## Testing the model
---

In [13]:
# Número de imagens classificadas corretamente
correct = 0
# Número total de imagens
total = 0

# Lista com as classes reais e classes preditas
true_test_list = []
pred_test_list = []

# Não é necessário calcular os gradientes.
with torch.no_grad():
    for inputs, labels in dataloader_test:
        
        inputs = inputs.to(DEVICE) 
        labels = labels.to(DEVICE) 
        
        # Calculo das saídas
        outputs = model(inputs)

        # A imagem é classificada de acordo com a maior saída
        _, predicted = torch.max(outputs.data, 1)

        # Atualiza o número de imagens
        total += labels.size(0)
        # Atualiza o número de classificações corretas
        correct += (predicted == labels).sum().item()

        true_test_list += list(labels.cpu())
        pred_test_list += list(predicted.cpu())

# Calcula a acurácia sobre o conjunto de testes
accuracy = correct / total

print(f'Acurácia de testes: {accuracy}')

Acurácia de testes: 0.5973


In [14]:
# Confusion matrix
conf_mat = metrics.confusion_matrix(true_test_list, pred_test_list)
print('\nConfusion matrix')
print(conf_mat)

# Classification report - Scikit-learn
class_rep = metrics.classification_report(true_test_list, 
                                          pred_test_list, 
                                          target_names=classes, 
                                          digits=4,
                                          zero_division=0)
print('\nClass. report')
print(class_rep)

# Accuracy
acc = metrics.accuracy_score(true_test_list, pred_test_list)
print('\n\nAcc.: {:.4f}'.format(acc))


Confusion matrix
[[704  28  63  18  32   9  11  14  76  45]
 [ 43 747   8  12   7  10  19  11  42 101]
 [122  10 440  67 122  60  91  50  15  23]
 [ 46  12  74 387 103 169 101  53  21  34]
 [ 41   8  95  71 532  43  97  88  17   8]
 [ 24   6  67 205  75 447  61  90   9  16]
 [ 16   8  53  72  65  36 710  12  12  16]
 [ 33   6  44  46 102  54  15 671   4  25]
 [124  36  16  22  18   8  14  13 710  39]
 [ 81 120  19  23  17  17  20  22  56 625]]

Class. report
              precision    recall  f1-score   support

       plane     0.5705    0.7040    0.6303      1000
         car     0.7615    0.7470    0.7542      1000
        bird     0.5006    0.4400    0.4683      1000
         cat     0.4193    0.3870    0.4025      1000
        deer     0.4958    0.5320    0.5133      1000
         dog     0.5240    0.4470    0.4825      1000
        frog     0.6234    0.7100    0.6639      1000
       horse     0.6553    0.6710    0.6630      1000
        ship     0.7380    0.7100    0.7238      

## Bibliografia
---
* PyTorch. Training a Classifier
    * https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html
* Microsoft. Train your image classifier model with PyTorch.
    * https://learn.microsoft.com/en-us/windows/ai/windows-ml/tutorials/pytorch-train-model

